# init spark session

In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .master("spark://spark-master:7077")
    .appName("test_hudi")
    .config("spark.driver.memory", "1g")
    .config("hive.metastore.uris", "thrift://hive-metastore:9083")
    .config("spark.sql.warehouse.dir", "hdfs://namenode:9000/user/hive/warehouse/default")
    .config("spark.jars", "hdfs://namenode:9000/user/hive/spark_jars/hudi-spark3.4-bundle_2.12-0.15.0.jar")
    .config("spark.sql.extensions", "org.apache.spark.sql.hudi.HoodieSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.hudi.catalog.HoodieCatalog")
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .config("spark.kryo.registrator", "org.apache.spark.HoodieSparkKryoRegistrar")
    .config("spark.executor.cores", 3)
    .config("spark.executor.memory", "6g")
    .config("spark.executor.instances", 2)
    .enableHiveSupport()
    .getOrCreate()
)

24/07/27 07:43:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/27 07:43:25 WARN TransportClientFactory: DNS resolution succeed for spark-master/172.18.0.9:7077 took 5007 ms


In [2]:
spark

# test hudi table

In [3]:
spark.sql(f"""
create database bizvault
location 'hdfs://namenode:9000/user/hive/warehouse/bizvault'
""").show()

++
||
++
++



In [24]:
spark.sql(f"""
create external table bizvault.test_hudi (key int, value string)
using hudi
location 'hdfs://namenode:9000/user/hive/warehouse/bizvault/test_hudi'
tblproperties(
    'hoodie.table.type'='mor',
    'hoodie.table.base.file.format'='parquet',
    'hoodie.datasource.meta.sync.enable'=true,
    'hoodie.datasource.hive_sync.mode'='hms',
    'hoodie.datasource.hive_sync.metastore.uris'='thrift://hive-metastore:9083',
    'hoodie.datasource.hive_sync.database'='bizvault',
    'hoodie.datasource.hive_sync.table'='test_hudi'
)
""").show()

++
||
++
++



24/07/27 09:00:46 WARN TableSchemaResolver: Could not find any data file written for commit, so could not get schema for table hdfs://namenode:9000/user/hive/warehouse/bizvault/test_hudi


In [25]:
spark.sql(f"""
insert into bizvault.test_hudi
values (1, "2")
""").show()

24/07/27 09:01:00 WARN TableSchemaResolver: Could not find any data file written for commit, so could not get schema for table hdfs://namenode:9000/user/hive/warehouse/bizvault/test_hudi
24/07/27 09:01:00 WARN TableSchemaResolver: Could not find any data file written for commit, so could not get schema for table hdfs://namenode:9000/user/hive/warehouse/bizvault/test_hudi
24/07/27 09:01:00 WARN AutoRecordKeyGenerationUtils$: Precombine field  will be ignored with auto record key generation enabled
24/07/27 09:01:04 WARN WriteMarkersFactory: Timeline-server-based markers are not supported for HDFS: base path hdfs://namenode:9000/user/hive/warehouse/bizvault/test_hudi.  Falling back to direct markers.
24/07/27 09:01:04 WARN WriteMarkersFactory: Timeline-server-based markers are not supported for HDFS: base path hdfs://namenode:9000/user/hive/warehouse/bizvault/test_hudi.  Falling back to direct markers.
24/07/27 09:01:04 WARN WriteMarkersFactory: Timeline-server-based markers are not supp

++
||
++
++



24/07/27 09:01:07 WARN HoodieSparkSqlWriterInternal: Closing write client


In [26]:
spark.sql(f"""
select * from bizvault.test_hudi
""").show()

+-------------------+--------------------+--------------------+----------------------+--------------------+---+-----+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|key|value|
+-------------------+--------------------+--------------------+----------------------+--------------------+---+-----+
|  20240727090100061|20240727090100061...|20240727090100061...|                      |441942c4-d488-4b0...|  1|    2|
+-------------------+--------------------+--------------------+----------------------+--------------------+---+-----+

